In [ ]:
library(tableone)
library(coxphw)
library(survival)
library(dplyr)
library(ggsurvfit)
library(cobalt)
library(ggplot2)
library(survminer)

# Import data

In [ ]:
df = read.csv('/home/jovyan/work/ROCKET AF/1A/masterfile_pos_final/masterfiles/df_wide_format_analysis.csv')

In [ ]:
head(df)

In [ ]:
options(scipen=999) # disable printing results in scientific notation

In [ ]:
head(df)

# Kaplan-Meier Analysis

In [ ]:
# Fit the Kaplan-Meier survival curves to the 'df' dataset stratified by the 'trt' variabale 

summary(survfit(Surv(time, status) ~ trt, data = df), times = 840)

In [ ]:


fit.obj <- survfit(Surv(time, status) ~ trt, 
                   data = df)

p1 <- ggsurvplot(
  fit.obj,                     
  data = df, 
  risk.table = FALSE,   # do not include a risk table in the plot
  pval = FALSE, # do not include p-values in the plot
  conf.int = TRUE, # do not display confidence intervals
  legend.labs = c("No Spiro","Spiro"), # customize legend labels
  font.legend = c(10), # set font size for the legend
  xlim = c(0,840), # set the x-axis limits
  break.time.by = 100,  # specify time breaks on the x-axis every 43 days
  xlab="Time in days", # customize the x-axis label
  ylab="Survival probability", # customize the y-axis label
  font.xlab = c(12), # set font size for the x-axis label
  font.ylab = c(12),  # set font size for the y-axis label
  ylim=c(0.65,1), # set the y-axis limits
  risk.table.height = 0.3, # specify the height of the risk table
  palette = c("slategrey","dodgerblue4")) # customize the color palette

p1

# Cox Proportional Hazards Model (without confounders)

In [ ]:
cox <- coxph(Surv(time, status) ~ as.factor(trt), data = df) 
summary(cox)

# Cox Proportional Hazards Model (adjusting for confounders using IPTW) - non stabilized weights

In [ ]:
# Estimation of propensity scores with a logistic model
ps.treatment <- glm(formula = trt==1 ~  age + eGFR + as.factor(gender),
                family = binomial(link = 'logit'),
                data = df)

summary(ps.treatment)

In [ ]:
#  Estimate propensity scores
df$ps_treatment <- predict(ps.treatment, df, type = "response")

In [ ]:
summary(df$ps_treatment[df$trt==1])
summary(df$ps_treatment[df$trt==0])

### Estimate the inverse probability weights 

In [ ]:
df$w_a <- ifelse(df$trt==1,
                 1/df$ps_treatment,
                 1/(1-df$ps_treatment))

In [ ]:
# Min, 25th percentile, median, mean, SD, 75th percentile, and max 
summary(df$w_a)
sd(df$w_a)

In [ ]:
# Create the histogram
hist(df$w_a, main = "IPTW Weights Distribution", xlab = "IPTW Weights")

###  Assessing the balance of covariates between treatment groups 

In [ ]:
covariates <- subset(df, select = c("age", "gender","eGFR"))
bal.tab(covariates, treat =df$trt, weights=df$w_a, un=TRUE)

###  Fit th weighted Cox proportional hazards model

In [ ]:
# Fit Weighted Cox Model 

fit1 <- coxphw(Surv(time, status) ~ as.factor(trt), 
               data = df,
               template = 'PH',
               caseweights = df$w_a)

summary(fit1)